# Downloadning data

In [1]:
import pandas as pd
test=pd.read_csv(r"F:\programming\fatsai\Google Patent Phrase Similarity Dataset\test.csv")

# Preprocessing data

In [2]:
test['input'] = 'TEXT1: ' + test.context + '; TEXT2: ' + test.target + '; ANC1: ' + test.anchor

In [3]:
from datasets import Dataset #using datasel library (hugging face) to run model
ds3 = Dataset.from_pandas(test)

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification #loading tokenizer and pretrained model
model_path = "my_deberta_model"
tokenizer = AutoTokenizer.from_pretrained(model_path) #loads tokenizer from pretrained model
model = AutoModelForSequenceClassification.from_pretrained(model_path) #loads wheights from model

In [5]:
def tok_func(x): #a function that back tokenizied inpots
    return tokenizer(x["input"])#now convert every tokenized vocab into numbers
tok_ds3 = ds3.map(tok_func, batched=True)

Parameter 'function'=<function tok_func at 0x000002C31479FBA0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/9232 [00:00<?, ? examples/s]

# Run and test 

In [6]:
import numpy as np # we use correlation as our metric 
def corr(x,y): #we build corrlation using numpy
    return np.corrcoef(x,y)[0][1] 

def corr_d(eval_pred): 
    return {'pearson': corr(*eval_pred)}# then convert it to the dictinery for transformers requirment 

In [7]:
from transformers import Trainer

trainer = Trainer(model=model, tokenizer=tokenizer)#Build model

In [8]:
predictions = trainer.predict(tok_ds3) # run and compute our prediction

  0%|          | 0/1154 [00:00<?, ?it/s]

In [9]:
preds = predictions.predictions.flatten() #convert prediction to flatted list
true_scores = test['score'].values  # our label

In [10]:
pearson_score = corr(preds, true_scores) #correlation error 
print("Test set Pearson correlation:",round(pearson_score,5))

Test set Pearson correlation: 0.81357
